In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
import pandas as pd
import re

In [35]:
fake_reviews_1 = pd.read_csv("/content/drive/MyDrive/data/fake_reviews.csv", names=['text'])
fake_reviews_2 = pd.read_csv("/content/drive/MyDrive/data/fake_reviews_continued.csv", names = ['text'])

real_reviews = pd.read_csv('/content/drive/MyDrive/data/sampled_dataset.csv', usecols=['text'])

In [36]:
fake_reviews = pd.concat([fake_reviews_1, fake_reviews_2], ignore_index=True)

In [37]:
fake_reviews = fake_reviews[fake_reviews['text'] != '0']

In [42]:
print(fake_reviews.iloc[4]['text'])

i have stayed at this hotel before and it was great, the staff are very nice and helpful, the location is great not too far from the city centre, but also out of the way so you don't have a lot of traffic noise, the room was very nice and


In [43]:
def post_process_text(text):
    # regex that splits the sentences
    sentences = re.split(r'(?<!\w\.\w)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)

    # Check if the last sentence meets the criteria for deletion as also described in Salminen et al.
    if len(sentences) > 0 and sentences[-1].endswith('.') and len(sentences[-1].split()) <= 3:
        return ' '.join(sentences[:-1])
    else:
        return text


In [44]:
fake_reviews['text'] = fake_reviews['text'].apply(post_process_text)

When continuing the generation process, I accidently skipped one prompt, meaning that only 4999 reviews are produced. To maintain a balanced sample, only 4999 real reviews will be used.

In [47]:
real_reviews = real_reviews.sample(n=4999, random_state=2023).reset_index()
real_reviews.drop(columns=['index'], inplace=True)

In [53]:
real_reviews['label'] = 1
fake_reviews['label'] = 0

In [54]:
result_df = pd.concat([real_reviews, fake_reviews], ignore_index=True)

In [55]:
result_df.to_csv('/content/drive/MyDrive/data/final_review_dataset.csv', sep=";", index=False)